In [1]:
using QuantumCumulants
using Symbolics
using SymbolicUtils


In [2]:
# Parameters
N = 1 #number of atoms
M = 2
kappa,Δ1,Δ2, q, gamma_par, etap, gamma_perp, kerr = cnumbers("κ Δ_c Δ_3 η γ_{∥} η_p γ_{⊥} χ")

wf, G, kappa_f = cnumbers("Δ_f G κ_f")


g(j) = cnumbers(Symbol("g_{$j}"))[1]
Δs(j) = cnumbers(Symbol("Δ_{$j}"))[1]

# Hilbertspace
hf = FockSpace(:cavity)
hf3 = FockSpace(:cavity)
ha = ⊗([NLevelSpace(Symbol(:atom,i),2) for i=1:N]...)
h = hf  ⊗ ha ⊗ hf3

ℋ(cavity) ⊗ ℋ(atom1) ⊗ ℋ(cavity)

In [31]:
# Operators
a = Destroy(h,:a, 1)
b = Destroy(h, :b, N+2)
sigma(i,j,k) = Transition(h,Symbol("σ_{$k}"),i,j,k+1)
sigmaz(k) = sigma(2,2,k)-sigma(1,1,k)
sigmap(k) = sigma(2,1,k)
sigmam(k) = sigma(1,2,k)

sigmam (generic function with 1 method)

In [32]:
H = Δ1*a'a+Δ2*b'b+1im*q*(b'-b)
for j=1:N
    H += Δs(j)/2*sigmaz(j)+1im*g(j)*(a'*sigmam(j)-a*sigmap(j))
end
#filter resonator
H += -kerr*a'a*b'b

(Δ_c*(a′*a)+Δ_3*(b′*b)+(0 + 1im)*η*(b′)+(0 - 1im)*η*(b)+(-1//2)*var"Δ_{1}"+(1//2)*var"Δ_{1}"*(σ_{1}22)+(1//2)*var"Δ_{1}"*(σ_{1}22)+(0 + 1im)*var"g_{1}"*(a′*σ_{1}12)+(0 - 1im)*var"g_{1}"*(a*σ_{1}21)+-χ*(a′*a*b′*b))

In [33]:
# Collapse operators

rates = Any[2*kappa]
rates = append!(rates, [gamma_par*(n+1) for i=1:N])
rates = append!(rates, [n*gamma_par+etap for i=1:N])
rates = append!(rates, [gamma_perp/2 for i=1:N])
#filter resonator
rates = append!(rates, [2*kappa_f])



J = Any[a]
J = append!(J, [sigmam(i) for i=1:N])
J = append!(J, [sigmap(i) for i=1:N])
J = append!(J, [sigmaz(i) for i=1:N])
#filter resonator
J = append!(J, [b]);

In [40]:
subdict = Dict([])

vec = ["\u2081", "\u2082", "\u2083", "\u2084", "\u2085", "\u2086", "\u2087", "\u2088", "\u2089"]


for i=1:N
    subs = vec[i]
    sm = "σ{$subs}⁻"
    sp = "σ{$subs}⁺"
    sz = "σ{$subs}ᶻ"
    
    #subdict1 = merge!(subdict1, Dict([(i)=>0]))
    #subdict1 = merge!(subdict1, Dict([(q)=>Symbol("0")]))
    
    for k=1:N
        subsk = vec[k]
        smk = "σ{$subsk}⁻"
        spk = "σ{$subsk}⁺"
        szk = "σ{$subsk}ᶻ"
        
        subdict = merge!(subdict, Dict([average(sigmam(k)*sigmap(i))=>Parameter(Symbol("⟨"*smk*sp*"⟩"))]))
        subdict = merge!(subdict, Dict([average(sigmap(k)*sigmap(i))=>Parameter(Symbol("⟨"*spk*sp*"⟩"))]))
        subdict = merge!(subdict, Dict([average(sigmap(k)*sigmam(i))=>Parameter(Symbol("⟨"*spk*sm*"⟩"))]))
        subdict = merge!(subdict, Dict([average(sigmam(k)*sigmam(i))=>Parameter(Symbol("⟨"*smk*sm*"⟩"))]))
       
        subdict = merge!(subdict, Dict([average(sigma(2,2,k)*sigmap(i))=>Parameter(Symbol("⟨"*szk*sp*"⟩"))/2+Parameter(Symbol("⟨"*sp*"⟩"))/2]))
        subdict = merge!(subdict, Dict([average(sigma(2,2,k)*sigmam(i))=>Parameter(Symbol("⟨"*szk*sm*"⟩"))/2+Parameter(Symbol("⟨"*sm*"⟩"))/2]))
        
        subdict = merge!(subdict, Dict([average(sigmap(k)*sigma(2,2,i))=>Parameter(Symbol("⟨"*spk*sz*"⟩"))/2+Parameter(Symbol("⟨"*spk*"⟩"))/2]))
        subdict = merge!(subdict, Dict([average(sigmam(k)*sigma(2,2,i))=>Parameter(Symbol("⟨"*smk*sz*"⟩"))/2+Parameter(Symbol("⟨"*smk*"⟩"))/2]))
        
        subdict = merge!(subdict, Dict([average(sigma(2,2,k)*sigma(2,2,i))=>1/4*(Parameter(Symbol("⟨"*szk*sz*"⟩"))+Parameter(Symbol("⟨"*sz*"⟩"))+Parameter(Symbol("⟨"*szk*"⟩"))+1)]))
    end
    
    subdict = merge!(subdict, Dict([average(sigmam(i))=>Parameter(Symbol("⟨"*sm*"⟩"))]))
    subdict = merge!(subdict, Dict([average(sigmap(i))=>Parameter(Symbol("⟨"*sp*"⟩"))]))
    subdict = merge!(subdict, Dict([average(sigma(2,2,i))=>(Parameter(Symbol("⟨"*sz*"⟩"))/2)+1/2]))
    
    
    
    
end
eqns = meanfield(a,H,J;rates=rates,order=M, simplify=true)

#careful here, this applies the approximations from the paper
#eqns = SymbolicUtils.simplify(Symbolics.substitute(eqns, subdict1), expand=true)

eq_n = SymbolicUtils.simplify(Symbolics.substitute(eqns, subdict), expand=true)

∂ₜ(⟨a⟩) = var"g_{1}"*var"⟨σ{₁}⁻⟩" + (0 - 1im)*Δ_c*⟨a⟩ + -κ*⟨a⟩ + (0 + 1im)*χ*⟨b′⟩*⟨a*b⟩ + (0 + 1im)*χ*⟨a⟩*⟨b′*b⟩ + (0 + 1im)*χ*⟨b⟩*⟨a*b′⟩ + (0 - 2im)*χ*⟨b′⟩*⟨a⟩*⟨b⟩
